In [2]:
# Required Modules
import numpy as np

In [15]:
class Links:

    def __init__(self, nodes, w):    # nodes = (s, d); is a tuple so as to not confuse s and d individually
        self.nodes = nodes
        self.lambda_tdc = np.ones(47, dtype = bool)

        time_slots1 = 8
        time_slots2 = 10 
        time_slots3 = 12
        self.lambda_q1 = np.ones(time_slots1, dtype = bool)    # for high sl
        self.lambda_q2 = np.ones(time_slots2, dtype = bool)    # for medium sl    
        self.lambda_q3 = np.ones(time_slots3, dtype = bool)    # for low sl

    def update_link(self, channel, ts):
        # call : links[index].update_link(channel, ts); where ts is timeslot
        #QSC = ["lambda_q1", "lambda_q2", "lambda_q3"]
        QSC = [1, 2, 3]
        if channel not in QSC or ts <= 0:
            print("Invalid Update request!")

        elif channel == 1 and ts <= len(self.lambda_q1):
            self.lambda_q1[ts-1] = False
            print("The updated resources are : ", self.lambda_q1)
        elif channel == 2 and ts <= len(self.lambda_q2):
            self.lambda_q2[ts-1] = False
            print("The updated resources are : ", self.lambda_q2)
        elif channel == 3 and ts <= len(self.lambda_q3):
            self.lambda_q3[ts-1] = False
            print("The updated resources are : ", self.lambda_q3)

        else:
            print("Invalid time slot value")

    def resources_available(self, channel, ts):
        if channel not in [1, 2, 3] or ts <= 0:
            print("Invalid Update request!")

        elif channel == 1 and ts <= len(self.lambda_q1):
            return self.lambda_q1[ts-1]
            
        elif channel == 2 and ts <= len(self.lambda_q2):
            return self.lambda_q2[ts-1]
            
        elif channel == 3 and ts <= len(self.lambda_q3):
            return self.lambda_q3[ts-1]

        else:
            print("Invalid time slot value")


    #def allocate_resources(self)
            
            
    def display_info(self):
        q1_count = np.count_nonzero(self.lambda_q1) # == True
        q2_count = np.count_nonzero(self.lambda_q2) #== True
        q3_count = np.count_nonzero(self.lambda_q3) #== True
        tdc_count = np.count_nonzero(self.lambda_tdc) #== True
        print(f"Link: Nodes ={self.nodes}, lambda_tdc_count={tdc_count}, lambda_q1_count={q1_count}, lambda_q2_count={q2_count}, lambda_q3_count={q3_count}")
        
    @classmethod
    def initialize_links(cls, edges):

        all_values = [nodes for row in edges for nodes in row[:2]]    # A list of all the values/nodes in first 2 columns : s & d
        m = min(all_values)
        M = max(all_values)
        numNodes = max(all_values) - min(all_values) + 1     # Correction for the convention in indexing of nodes in the data of edges
    
        links = np.zeros([M+1, M+1], dtype = object)    # Will have redundant entries. M is the highest node index
        #link_indices = np.zeros_like(links, dtype = bool)

        #links = {}
        
        for (s, d, w) in edges:

            nodes = (s, d)
            link = cls(nodes, w)
            
            links[nodes] = link
            links[d, s] = link
            
            #print("link info")
            #link.display_info() # Working

            #print("links[nodes] info")
            #links[nodes].display_info() # Working

            #if min(all_values) == 0:
             #   (S, D) = (s, d)
            #else :
             #   (S, D) = (s-1, d-1)
            # (S, D) = (s-m, d-m)    # However, we won't be able to access the links with their indices in this case
            #links[s, d] = link
            #links[d, s] = link
            #link_indices(s, d) = True
            #link_indices(d, s) = True
                    
        indices = links != 0    # A mask that holds the location of each link

        return (links, indices)

    @classmethod
    def FF(cls, path, cr):
        # First check for availability of resources
        # We've not considered ASLC and Resource usage extent here]
    
        sl = cr.sl
        print(sl)
        n_ts = {"high" : 8, "medium" : 10, "low" : 12}
        available_ts = [True] * n_ts[sl]

        #print("available_ts: ", available_ts)

        for s, d in zip(path, path[1:]):    # Taking consecutive pairs of nodes
            if sl == "high":
                band = links[s, d].lambda_q1
                print(f"lambda_q1 : {band}, link : {s, d}")
            elif sl == "medium":
                band = links[s, d].lambda_q2
                print(f"lambda_q2 : {band}, link : {s, d}")
            else:
                band = links[s, d].lambda_q3
                print(f"lambda_q3 : {band}, link : {s, d}")
                
            available_ts = [a and b for a, b in zip(available_ts, band)]

        return available_ts

In [6]:
#print(indices)

In [7]:
#print(links)

In [8]:
##for index in indices: # ERROR : Because indices isn't an array. See below cell for correction
  ##  links[index].display_info()